<a href="https://colab.research.google.com/github/mahfujkhan02/Api-meals-food/blob/main/Test1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install selenium openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.3/486.3 kB 22.7 MB/s eta 0:00:00


In [4]:
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from datetime import datetime
import openpyxl

In [13]:
EXCEL_FILE = 'Covid-19 Data Excel Sheet.xlsx'
def read_keywords_from_excel():
    today = datetime.now().strftime("%A")
    workbook = openpyxl.load_workbook(EXCEL_FILE)
    if today in workbook.sheetnames:
        sheet = workbook[today]
        keywords = [sheet.cell(row=i, column=1).value for i in range(2, sheet.max_row + 1)]
        return keywords, workbook, sheet
    else:
        raise ValueError(f"No sheet found for {today}")

def write_results_to_excel(sheet, results):
    for i, (keyword, shortest, longest) in enumerate(results, start=2):
        sheet.cell(row=i, column=2).value = shortest
        sheet.cell(row=i, column=3).value = longest

In [6]:
def get_search_suggestions(driver, keyword):
    driver.get("https://www.google.com")
    search_box = driver.find_element(By.NAME, "q")
    search_box.send_keys(keyword)
    search_box.send_keys(Keys.RETURN)


    driver.implicitly_wait(2)
    suggestions = driver.find_elements(By.CSS_SELECTOR, "ul[role='listbox'] li")
    suggestions_text = [s.text for s in suggestions if s.text.strip()]

    if suggestions_text:
        longest = max(suggestions_text, key=len)
        shortest = min(suggestions_text, key=len)
    else:
        longest = shortest = "No suggestions found"
    return shortest, longest

In [7]:
def main():
    driver = setup_driver()
    keywords, workbook, sheet = read_keywords_from_excel()
    results = []

    for keyword in keywords:
        print(f"Searching for: {keyword}")
        shortest, longest = get_search_suggestions(driver, keyword)
        print(f"Shortest: {shortest}, Longest: {longest}")
        results.append((keyword, shortest, longest))

    write_results_to_excel(sheet, results)
    workbook.save(EXCEL_FILE)
    workbook.close()
    driver.quit()
    print("Process completed successfully!")

In [9]:
from google.colab import files
uploaded = files.upload()

Saving Covid-19 Data Excel Sheet.xlsx to Covid-19 Data Excel Sheet.xlsx
